In [7]:
import numpy as np
import numpy.matlib
import pickle
import matplotlib.pyplot as plt
import scipy.linalg as sl
import time
import pathlib
import nbimporter  # I installed it for importing an .ipynb file into another .ipynb like the pilot_assignemnt
import os

from datetime import datetime
#import pilot_assignment
#import APLocation_Generation
#import functionRlocalscattering
#------------------------------
#import Power_optimizers



from scipy.optimize import minimize
from scipy.optimize import differential_evolution
import numpy.linalg as l 


import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
import math

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
import time
#import numpy.linalg as nl 


import tensorflow.keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as Kr
from tensorflow.keras import initializers

import pandas as pd
import random
import cv2
import os
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score


from tensorflow import expand_dims
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, Lambda
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD


from __future__ import print_function
import keras
import tensorflow.keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras import initializers

from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import np_utils 

import time

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import mnist

In [32]:


def uniform_quantization(y, R, b):
    """
    Perform uniform quantization on the vector y with b bits, using a radius R.

    Parameters:
    y (numpy array): The vector to be quantized
    R (float): The radius for quantization
    b (int): Number of bits for quantization

    Returns:
    numpy array: Quantized vector
    """
    # Number of quantization levels
    L = 2 ** b
    
    # Quantization step size
    delta = 2 * R / (L - 1)
    
    # Quantize the vector
    y_clipped = np.clip(y, -R, R)  # Ensure values are within [-R, R]
    quantized = np.round(y_clipped / delta) * delta
    
    return quantized

def LAQ(x, x_prev, b):
    """
    Quantize the difference between vectors x and x_prev.

    Parameters:
    x (numpy array): Current vector
    x_prev (numpy array): Previous vector
    b (int): Number of bits for quantization

    Returns:
    numpy array: Quantized difference vector
    """
    # Compute the difference vector
    y = x - x_prev
    
    # Compute the radius R
    R = np.max(np.abs(y))
    
    # Perform uniform quantization
    y_quantized = uniform_quantization(y, R, b)
    err = y_quantized - y
    x_q = y_quantized + x_prev
    return x_q, y_quantized, err

# Example usage
x = np.array([1.2, -0.8, 0.5, -1.0])
x_prev = np.array([1.0, -1.0, 0.4, -0.8])
b = 2  # Number of bits for quantization

# Quantize the difference
x_q, y_quantized, error = LAQ(x, x_prev, b)
print("Quantized + prev:", x_q)
np.max(np.abs(error))






def AQUILA(x, x_prev, d):
    """
    Quantize the difference between vectors x and x_prev.

    Parameters:
    x (numpy array): Current vector
    x_prev (numpy array): Previous vector
    b (int): Number of bits for quantization

    Returns:
    numpy array: Quantized difference vector
    """
    # Compute the difference vector
    y = x - x_prev
    
    # Compute the radius R
    R = np.max(np.abs(y))
    b = np.floor(np.log2(1 + R*np.sqrt(d)/l.norm(y, 2) ))
    
    # Perform uniform quantization
    y_quantized = uniform_quantization(y, R, b)
    err = y_quantized - y
    x_q = y_quantized + x_prev
    return x_q, y_quantized, err, b

# Example usage
x = np.array([1.2, -0.8, 0.5, -1.0])
x_prev = np.array([1.0, -1.0, 0.4, -0.8])


# Quantize the difference
x_q, y_quantized, error, b =  AQUILA(x, x_prev, x.shape[0])
print("Quantized + prev:", x_q)
print(b)
np.max(np.abs(error))


Quantized + prev: [ 1.26666667 -0.73333333  0.53333333 -1.06666667]
Quantized + prev: [ 1.  -1.   0.4 -0.8]
1.0


0.19999999999999996

In [33]:
#non-iid
K = 40



# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize data
num_clients= K


#------------------------------DATASETS--------------------------------------------------
## Split the data into non-overlapping portions for each client
# Load CIFAR-10 dataset


#------------------------------DATASETS--------------------------------------------------
## Split the data into non-overlapping portions for each client
client_data = []
samples_per_client = len(x_train) // num_clients
#for i in range(num_clients):
#    start = i * samples_per_client
#    end = (i + 1) * samples_per_client
#    x_client = x_train[start:end]
#    y_client = y_train[start:end]
#    client_data.append((x_client, y_client))

# Assume x_train and y_train are numpy arrays, and num_clients is an integer
# Assume x_train and y_train are numpy arrays, and num_clients is an integer
np.random.seed(42)  # For reproducibility

# Shuffle the data to start with a random distribution
indices = np.arange(len(x_train))
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

# Group data by class for controlled allocation
class_indices = {}
for label in np.unique(y_train):
    class_indices[label] = list(np.where(y_train == label)[0])  # Convert to list for easy pop operations

# Split data for each client with varying class proportions
client_data = []
num_classes = len(class_indices)
samples_per_client = len(x_train) // num_clients  # Ensures equal samples per client

for i in range(num_clients):
    x_client = []
    y_client = []

    # Determine class proportions for this client
    proportions = np.random.dirichlet(alpha=[0.4] * num_classes)

    # Track the number of samples allocated to this client
    allocated_samples = 0

    while allocated_samples < samples_per_client:
        for label in range(num_classes):
            # Calculate how many samples to take for this class based on proportions
            num_samples = int(proportions[label] * samples_per_client)

            # Adjust to ensure we fill the client's quota exactly
            if allocated_samples + num_samples > samples_per_client:
                num_samples = samples_per_client - allocated_samples

            # Ensure we don't sample more than available for the class
            num_samples = min(num_samples, len(class_indices[label]))

            # Select the samples and add them to the client's data
            class_samples = class_indices[label][:num_samples]
            x_client.extend(x_train[class_samples])
            y_client.extend(y_train[class_samples])

            # Remove used samples from class
            class_indices[label] = class_indices[label][num_samples:]

            # Update allocated sample count
            allocated_samples += len(class_samples)

            # If the client has enough samples, break the loop
            if allocated_samples >= samples_per_client:
                break

    # Shuffle the client's data to randomize order
    x_client = np.array(x_client)
    y_client = np.array(y_client)
    client_indices = np.arange(len(x_client))
    np.random.shuffle(client_indices)
    x_client = x_client[client_indices]
    y_client = y_client[client_indices]

    client_data.append((x_client, y_client))
# Plot distribution for a few clients
def plot_client_data_distribution(client_data, client_labels, num_clients, num_samples=5):
    """
    Plot sample images from each client's dataset to visualize the distribution.
    
    :param client_data: List of data arrays for each client.
    :param client_labels: List of label arrays for each client.
    :param num_clients: Number of clients to visualize.
    :param num_samples: Number of samples to display per client.
    """
    plt.figure(figsize=(15, 3 * num_clients))
    
    for client_id in range(num_clients):
        data = client_data[client_id]
        labels = client_labels[client_id]
        
        for i in range(num_samples):
            plt.subplot(num_clients, num_samples, client_id * num_samples + i + 1)
            plt.imshow(data[i])
            plt.title(f"Client {client_id}, Label: {labels[i][0]}")
            plt.axis('off')
    
    plt.tight_layout()
    plt.show()
#----------------------------------------------------------------------------------------------

# client_data now contains non-IID datasets for each client

# Display some information about the distribution
for i, (data, labels) in enumerate(client_data):
    print(f"Client {i}: {len(data)} samples")
    unique, counts = np.unique(labels, return_counts=True)
    label_distribution = dict(zip(unique, counts))
    print(f"Label distribution: {label_distribution}\n")

# Plot distribution for a few clients
def plot_client_data_distribution(client_data, client_labels, num_clients, num_samples=5):
    """
    Plot sample images from each client's dataset to visualize the distribution.
    
    :param client_data: List of data arrays for each client.
    :param client_labels: List of label arrays for each client.
    :param num_clients: Number of clients to visualize.
    :param num_samples: Number of samples to display per client.
    """
    plt.figure(figsize=(15, 3 * num_clients))
    
    for client_id in range(num_clients):
        data = client_data[client_id]
        labels = client_labels[client_id]
        
        for i in range(num_samples):
            plt.subplot(num_clients, num_samples, client_id * num_samples + i + 1)
            plt.imshow(data[i])
            plt.title(f"Client {client_id}, Label: {labels[i][0]}")
            plt.axis('off')
    
    plt.tight_layout()
    plt.show()
#----------------------------------------------------------------------------------------------

Client 0: 1250 samples
Label distribution: {0: 22, 1: 328, 3: 249, 4: 64, 5: 7, 6: 147, 8: 194, 9: 239}

Client 1: 1250 samples
Label distribution: {0: 72, 1: 1, 3: 35, 4: 61, 5: 1, 6: 126, 7: 62, 8: 892}

Client 2: 1250 samples
Label distribution: {0: 170, 1: 241, 2: 1, 3: 31, 4: 203, 5: 191, 6: 314, 7: 7, 8: 76, 9: 16}

Client 3: 1250 samples
Label distribution: {0: 82, 1: 98, 2: 52, 3: 3, 4: 492, 5: 389, 7: 17, 8: 6, 9: 111}

Client 4: 1250 samples
Label distribution: {2: 49, 3: 1, 4: 10, 5: 116, 6: 307, 7: 13, 8: 723, 9: 31}

Client 5: 1250 samples
Label distribution: {0: 229, 1: 113, 2: 180, 3: 47, 4: 159, 6: 100, 7: 10, 8: 320, 9: 92}

Client 6: 1250 samples
Label distribution: {1: 222, 2: 207, 3: 14, 4: 3, 5: 15, 6: 500, 8: 64, 9: 225}

Client 7: 1250 samples
Label distribution: {0: 23, 1: 3, 2: 84, 3: 21, 4: 24, 5: 222, 6: 233, 7: 336, 8: 201, 9: 103}

Client 8: 1250 samples
Label distribution: {2: 290, 3: 26, 5: 324, 6: 345, 7: 143, 8: 68, 9: 54}

Client 9: 1250 samples
Label 

In [34]:
# Define neural network architecture
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model
# Define the number of clients and local epochs
num_clients= K
#local_epochs = 1
local_epochs0 = 5 #local_epochs FedAvg
local_epochs = 5
epoch_increase = 0
b_j = 10
# Federated Averaging algorithm
num_global_iterations = 50
k_0 = -100 #num_global_iterations +1

learning_rate = 1e-3
optimizer = SGD(learning_rate=learning_rate)
#optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1)

#optimizer = tf.keras.optimizers.Adadelta(learning_rate=1.0)
#optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-4) #


## Split the data into non-overlapping portions for each client
#client_data = []
#samples_per_client = len(x_train) // num_clients
#for i in range(num_clients):
#    start = i * samples_per_client
#    end = (i + 1) * samples_per_client
#    x_client = x_train[start:end]
#    y_client = y_train[start:end]
#    client_data.append((x_client, y_client))

    
    
##Lambda0 = np.random.rand(num_clients)
#Lambda0 = np.abs(np.random.randn(num_clients))
#LAmbda = np.zeros(num_clients)

#for i in range(num_clients):
#    LAmbda[i] = (1 - Lambda0[i]/np.max(Lambda0) + 0.001)
#Lambda = LAmbda/np.sum(LAmbda)
print('Lambda:', Lambda)
#print('sum lambda:', np.sum(Lambda))
# Initialize the global model
global_model = create_model()


# Compile the global model
global_model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
#global_model_FedAvg = global_model 
global_model_FedAvg = create_model()
global_model_FedAvg.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
global_weights = global_model.get_weights()
B_q = np.zeros([8,num_global_iterations])

#------------- FedAvg benchmark----------------------


global_model_FedAvg.set_weights(global_weights)
global_acc_list = []
global_loss_list = []

global_acc_list_FedAvg = []
global_loss_list_FedAvg = []

#----------- Initial values:
initial_loss, initial_accuracy = global_model.evaluate(x_test, y_test)
print(f'Initial Test Loss: {initial_loss:.4f}')
print(f'Initial Test Accuracy: {initial_accuracy:.4f}')

global_acc_list_FedAvg.append(initial_accuracy)
global_acc_list.append(initial_accuracy)

Lambda: <class 'tensorflow.python.keras.layers.core.Lambda'>
313/313 [==============================] - 2s 4ms/step - loss: 2.3323 - accuracy: 0.1023
Initial Test Loss: 2.3330
Initial Test Accuracy: 0.1003


In [36]:
Bits = np.zeros([num_clients, num_global_iterations])
percentage = np.zeros([num_clients, num_global_iterations])
#-----------------------------------------------------
t_start=time.time()
dimension= 462410
Previous_diff_local_global_weight = np.zeros([num_clients, dimension]).T

for k in range(num_global_iterations):
    print(f"Global Iteration {k + 1}/{num_global_iterations}")

    # List to store client models for averaging
    client_models = []
    client_models_FedAvg = []
    

    for i in range(num_clients):
        print(f"Client {i + 1}/{num_clients}")
        x_local, y_local = client_data[i]

        # Create a copy of the global model for each client
        local_model = create_model()
        local_model.set_weights(global_weights)#global_model.get_weights())
        #----------------------- FedAvg benchmark--------------------------------
        local_model_FedAvg = create_model()
        local_model_FedAvg.set_weights(global_model_FedAvg.get_weights())
        local_model_FedAvg .compile(optimizer=optimizer,
                            loss='sparse_categorical_crossentropy',
                            metrics=['accuracy'])
        local_model_FedAvg .fit(x_local, y_local, epochs=local_epochs0, verbose=0)
        
            
        #----------------------------------------------------
        # Compile and train the local model
        local_model.compile(optimizer=optimizer,
                            loss='sparse_categorical_crossentropy',
                            metrics=['accuracy'])
        
        local_model.fit(x_local, y_local, epochs=local_epochs , verbose=0)

        if k<= k_0:
            local_model.set_weights(local_model_FedAvg.get_weights())
#--------------------------------------------------- Quantization -----------------------------------------
        if k>k_0:
        # Quantize the local model weights
        #quantized_weights = [np.round(w * 127) / 127 for w in local_model.get_weights()]
            diff_local_global_weight = np.array(local_model.get_weights())#-np.array(global_weights) 
            local_weight_shape_0 = diff_local_global_weight.shape[0]
            quantized_diff_local_global_weight = diff_local_global_weight
            err = np.zeros([local_weight_shape_0 ,1])
            b_p = np.zeros([local_weight_shape_0 ,1])
            b_q =  np.zeros([local_weight_shape_0 ,1])
            x_previous = np.zeros([local_weight_shape_0 ,1])
            x_all_original = np.zeros([local_weight_shape_0 ,1])
            p10 = 0
            x_all = 0
            x = np.zeros([local_weight_shape_0,1])
            Quantized_diff_local_global_weight = []
            for h in range(local_weight_shape_0):
                x_position = 0
                if h==0:
                    quantized_diff_local_global_weight[h] = quantized_diff_local_global_weight[h].reshape(27*32,)
                    #Quantized_diff_local_global_weight = np.concatenate((Quantized_diff_local_global_weight,quantized_diff_local_global_weight[h]))
               ## quantized_diff_local_global_weight[h], tau, err[h], r = Quantization_NNs(bits,R,diff_local_global_weight[h])
                if h==2:
                    quantized_diff_local_global_weight[h] = quantized_diff_local_global_weight[h].reshape(7200*64,)
                
                if h in (1, 3, 5):
                    quantized_diff_local_global_weight[h] = quantized_diff_local_global_weight[h].reshape(quantized_diff_local_global_weight[h].shape[0],)
                
                #if h==6:
                   # quantized_diff_local_global_weight[h] = quantized_diff_local_global_weight[h].reshape(32,1)
                    
                if h==4:
                    quantized_diff_local_global_weight[h] = quantized_diff_local_global_weight[h].reshape(10*64,)
                
               #### x_previous[h] = np.max(abs(quantized_diff_local_global_weight[h]))
                ####x_position = np.where(np.abs(quantized_diff_local_global_weight[h]) == 
                ####                      max(np.abs(quantized_diff_local_global_weight[h])))
                ####x_temp=quantized_diff_local_global_weight[h] 
                ####x_all_original[h] = x_temp[x_position[0][0]] 
                #print(x[b[0][0]])
            
            #print(x_previous.T)
            ####x_all = np.max(x_previous)
           #### x_position_all = np.where(np.abs(x_previous) == max(np.abs(x_previous)))
            ####x_all_original1 = x_all_original[x_position_all[0][0]]
           #### print('x_all_original1:', x_all_original1)
            
           #### #print(x_previous.T)
            ####x_all = np.max(x_previous)
            #print('x_all:', x_all)
            
            for h in range(local_weight_shape_0):
                Quantized_diff_local_global_weight = np.concatenate((Quantized_diff_local_global_weight,quantized_diff_local_global_weight[h]))
           
            
            Quantized_diff_local_global_weight_for_me = Quantized_diff_local_global_weight
            print('max:',np.max(Quantized_diff_local_global_weight), '----min:',np.min(Quantized_diff_local_global_weight))
           
        # print('min:',np.min(Quantized_diff_local_global_weight))
        
            #nmax = 10000#min(2000*(2+k), 50000)
            #nmin = nmax
           # print( 'min of absolute value:',np.min(np.abs(Quantized_diff_local_global_weight)))
            #Quantized_diff_local_global_weight, max_model, min_model, err = adaptive_signQuant(Quantized_diff_local_global_weight, Quantized_diff_local_global_weight.shape[0], nmax,nmin)
            
            Quantized_diff_local_global_weight, y_q, err, b_AQUILA= AQUILA(Quantized_diff_local_global_weight, 
                                                                           Previous_diff_local_global_weight[:,i], dimension)
 
            Bits[i,k] = (dimension )* (b_AQUILA)
            Quantized_diff_local_global_weight1 = Quantized_diff_local_global_weight
            Previous_diff_local_global_weight[:,i] = np.copy(Quantized_diff_local_global_weight)
            print('b:', b_AQUILA)
           
            print('max error:', np.max(np.abs(err)))# - np.sum(Quantized_diff_local_global_weight1))
            #print('sum 1r:', np.sum(Quantized_diff_local_global_weight1))
            #print('max_model:', max_model)
            #print('min_model:', min_model)
            #print('max_model:', max_model, 'min_model:', min_model, 'x_min_me:', x_min_me)
            #percentage[i,k] = 100*(n_min+n_max)/462410
            #print('percentage %:', percentage[i,k] )
            #------------------ Update the local model with quantized weights:-----------------------------
            dim_h = 0
            for h in range(local_weight_shape_0):
                if h==0:
                    quantized_diff_local_global_weight[h] = Quantized_diff_local_global_weight[0:27*32]
                    quantized_diff_local_global_weight[h] = quantized_diff_local_global_weight[h].reshape(3, 3, 3, 32)
                    dim_h += 27*32
                    
                    
                if h==1:
                    quantized_diff_local_global_weight[h] = Quantized_diff_local_global_weight[dim_h: dim_h+32]
                    quantized_diff_local_global_weight[h] = quantized_diff_local_global_weight[h].reshape(32,)
                    dim_h += 32
                if h==2: 
                    quantized_diff_local_global_weight[h] = Quantized_diff_local_global_weight[dim_h: dim_h+7200*64]
                    quantized_diff_local_global_weight[h] = quantized_diff_local_global_weight[h].reshape(7200, 64)
                    dim_h+= 7200*64
                if h==3:
                    quantized_diff_local_global_weight[h] = Quantized_diff_local_global_weight[dim_h: dim_h+64]
                    quantized_diff_local_global_weight[h] = quantized_diff_local_global_weight[h].reshape(64,)
                    dim_h += 64
            
                #if h==6:
                 #   quantized_diff_local_global_weight[h] = quantized_diff_local_global_weight[h].reshape(64,10)
                    
                if h==4:
                    quantized_diff_local_global_weight[h] = Quantized_diff_local_global_weight[dim_h: dim_h+640]
                    quantized_diff_local_global_weight[h] = quantized_diff_local_global_weight[h].reshape(64,10)
                    dim_h += 640
              
                if h==5:
                    quantized_diff_local_global_weight[h] = Quantized_diff_local_global_weight[dim_h: dim_h+10]
                    quantized_diff_local_global_weight[h] = quantized_diff_local_global_weight[h].reshape(10,)
                    dim_h += 10
            
            
            local_model.set_weights(quantized_diff_local_global_weight)# + np.array(global_weights))
        #else:
           # local_model.set_weights()
            
        # Add the trained and quantized local model to the list
        client_models.append(local_model)
        client_models_FedAvg.append(local_model_FedAvg)
        #B_q[:,k] += b_q
            
        #---------------------------FedAvg benchmark---------------------------------
    #print('nmax:', nmax)
    global_weights_FedAvg = np.mean([model0.get_weights() for model0 in client_models_FedAvg], axis=0)
    global_model_FedAvg.set_weights(global_weights_FedAvg)

#------------------------------------------------------------------------------------------------------------
    # Calculate the global model's new weights as the average of the client models
    if k <= k_0:
        global_model.set_weights(global_weights_FedAvg)
        global_weights = global_weights_FedAvg
    else:
        global_weights = np.mean([model.get_weights() for model in client_models], axis=0)
        global_model.set_weights(global_weights)
        #local_epochs = min(12, local_epochs + epoch_increase)
        #local_epochs += epoch_increase
        #local_epochs0 = local_epochs
        #local_epochs = max(1, local_epochs)
    
    if k == k_0:
        local_epochs += epoch_increase
        local_epochs0 = local_epochs
   # B_q[:,k] = b_q
#----------------------------------------------------------------------------
    # Evaluate the global model on the test data
    print('------------------------------------------------------------------------------------------------------')
    
    loss, accuracy = global_model.evaluate(x_test, y_test)
    print(f"Global Test loss: {loss:.4f}, Global Test accuracy: {accuracy:.4f}")
    global_acc_list.append(accuracy)
    global_loss_list.append(loss)
   # print('Lambda:',Lambda)
# The final global model is stored in 'global_model'
#-------------------------------------Evaluate the FedAvg global model on the test data----------------------
    print('------------------------------ ------FedAvg Benchmark------------------------------------------------')
    loss_FedAvg, accuracy_FedAvg = global_model_FedAvg.evaluate(x_test, y_test)
    print(f"_FedAvg Global Test loss: {loss_FedAvg:.4f}, _FedAvg Global Test accuracy: {accuracy_FedAvg:.4f}")
    print('------------------------------------------------------------------------------------------------------')
    global_acc_list_FedAvg.append(accuracy_FedAvg)
    global_loss_list_FedAvg.append(loss_FedAvg)
    #----------------
   # plt.figure(figsize=(7, 3))
   # plt.plot(global_acc_list)    
   # plt.plot(global_acc_list_FedAvg)
   # plt.legend(['Quant', 'FedAvg'])
   # plt.grid(True, which='major')   
    
plt.figure(figsize=(7, 3))
plt.plot(global_acc_list )    
plt.plot(global_acc_list_FedAvg)
plt.legend(['Quant', 'FedAvg'])
plt.grid(True, which='major')    
t_stop=time.time()            
print('simulation time:', t_stop- t_start)            
            
            

Global Iteration 1/50
Client 1/40


C:\Users\afmb\AppData\Local\Temp\ipykernel_12836\1875016897.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  diff_local_global_weight = np.array(local_model.get_weights())#-np.array(global_weights)


max: 0.28410008549690247 ----min: -0.28185465931892395
b: 4.0
max error: 0.018940005699793483
Client 2/40
max: 0.29788845777511597 ----min: -0.2818411886692047
b: 4.0
max error: 0.019859230518341042
Client 3/40
max: 0.28409862518310547 ----min: -0.281859427690506
b: 4.0
max error: 0.01893990834554038
Client 4/40
max: 0.29057776927948 ----min: -0.28217896819114685
b: 4.0
max error: 0.01937185128529867
Client 5/40
max: 0.2961631417274475 ----min: -0.2818531394004822
b: 4.0
max error: 0.019744209448496475
Client 6/40
max: 0.28410008549690247 ----min: -0.28186309337615967
b: 4.0
max error: 0.018940005699793483
Client 7/40
max: 0.28637999296188354 ----min: -0.28185394406318665
b: 4.0
max error: 0.019091999530792214
Client 8/40
max: 0.2910829782485962 ----min: -0.2818540036678314
b: 4.0
max error: 0.019405531883239735
Client 9/40
max: 0.2877408266067505 ----min: -0.2818524241447449
b: 4.0
max error: 0.01918272177378333
Client 10/40
max: 0.28410008549690247 ----min: -0.28185757994651794
b: 4.

C:\Users\afmb\Anaconda3\lib\site-packages\numpy\core\_methods.py:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


------------------------------------------------------------------------------------------------------
313/313 [==============================] - 1s 4ms/step - loss: 2.3371 - accuracy: 0.0952
Global Test loss: 2.3371, Global Test accuracy: 0.0952
------------------------------ ------FedAvg Benchmark------------------------------------------------
313/313 [==============================] - 2s 5ms/step - loss: 2.3237 - accuracy: 0.1080
_FedAvg Global Test loss: 2.3230, _FedAvg Global Test accuracy: 0.1078
------------------------------------------------------------------------------------------------------
Global Iteration 2/50
Client 1/40
max: 0.2812730669975281 ----min: -0.27202901244163513
b: 3.0
max error: 0.005276144445053345
Client 2/40
max: 0.28418222069740295 ----min: -0.2769489288330078
b: 3.0
max error: 0.005348739027976987
Client 3/40
max: 0.2816377878189087 ----min: -0.2707963287830353
b: 3.0
max error: 0.005276117319729004
Client 4/40
max: 0.2818705439567566 ----min: -0.2740

...

max: 0.3280760943889618 ----min: -0.2950492799282074
b: 5.0
max error: 0.00017086642387035157
Client 30/40
max: 0.32923468947410583 ----min: -0.2949506342411041
b: 4.0
max error: 0.00018113155588460415
Client 31/40
max: 0.31975266337394714 ----min: -0.2946292459964752
b: 4.0
max error: 0.00018902990818065813
Client 32/40
max: 0.3216637074947357 ----min: -0.29469555616378784
b: 5.0
max error: 0.00014031897177176934
Client 33/40
max: 0.32895800471305847 ----min: -0.29393672943115234
b: 4.0
max error: 0.0001682454768288746
Client 34/40
max: 0.31989291310310364 ----min: -0.29410478472709656
b: 4.0
max error: 0.00023214701062513905
Client 35/40
max: 0.33917251229286194 ----min: -0.29649364948272705
b: 5.0
max error: 9.024890077413868e-05
Client 36/40
max: 0.3310357332229614 ----min: -0.2945142984390259
b: 5.0
max error: 0.0001294072843206033
Client 37/40
max: 0.32626107335090637 ----min: -0.29381874203681946
b: 5.0
max error: 9.448812078642274e-05
Client 38/40
max: 0.3254329264163971 ----mi

max: 0.3224649131298065 ----min: -0.29620563983917236
b: 5.0
max error: 0.00014554406620960816
Client 15/40
max: 0.3255322575569153 ----min: -0.29508280754089355
b: 5.0
max error: 0.00010996273087689754
Client 16/40
max: 0.32235997915267944 ----min: -0.2943044602870941
b: 4.0
max error: 0.00015973387445664686
Client 17/40
max: 0.32456737756729126 ----min: -0.2938552796840668
b: 4.0
max error: 0.0002908969624841034
Client 18/40
max: 0.32442864775657654 ----min: -0.2950938045978546
b: 4.0
max error: 0.00022962660245591602
Client 19/40
max: 0.33108457922935486 ----min: -0.2978927791118622
b: 4.0
max error: 0.00015862528630908876
Client 20/40
max: 0.3231505751609802 ----min: -0.2949829399585724
b: 5.0
max error: 0.00012052448483886816
Client 21/40
max: 0.3239348232746124 ----min: -0.29396793246269226
b: 5.0
max error: 0.00010809832082437059
Client 22/40
max: 0.3214174509048462 ----min: -0.29320091009140015
b: 5.0
max error: 0.00011162968634289986
Client 23/40
max: 0.3258994221687317 ----mi

max: 0.32915112376213074 ----min: -0.2972562611103058
b: 5.0
max error: 0.0001721869310624454
Client 30/40
max: 0.33091166615486145 ----min: -0.2969001233577728
b: 5.0
max error: 0.00012709271699614874
Client 31/40
max: 0.32163578271865845 ----min: -0.2967566251754761
b: 4.0
max error: 0.00017413088185999452
Client 32/40
max: 0.323885053396225 ----min: -0.2968701124191284
b: 4.0
max error: 0.00014083373591956523
Client 33/40
max: 0.32930344343185425 ----min: -0.2960472106933594
b: 4.0
max error: 0.00014207092421295942
Client 34/40
max: 0.32130253314971924 ----min: -0.29556435346603394
b: 4.0
max error: 0.00017417719630619024
Client 35/40
max: 0.3394148051738739 ----min: -0.2986454963684082
b: 5.0
max error: 0.0002071571419631447
Client 36/40
max: 0.3324281871318817 ----min: -0.29673054814338684
b: 5.0
max error: 0.0001354480320333966
Client 37/40
max: 0.32588478922843933 ----min: -0.29569724202156067
b: 5.0
max error: 7.13267832810967e-05
Client 38/40
max: 0.32822027802467346 ----min: 

max: 0.3236970007419586 ----min: -0.2954186201095581
b: 5.0
max error: 0.0001516181228843486
Client 15/40
max: 0.32507583498954773 ----min: -0.29698437452316284
b: 5.0
max error: 9.587754799333589e-05
Client 16/40
max: 0.32375648617744446 ----min: -0.2959545850753784
b: 5.0
max error: 9.581815738938463e-05
Client 17/40
max: 0.32629090547561646 ----min: -0.2955804467201233
b: 4.0
max error: 0.00023370199284213165
Client 18/40
max: 0.3258129358291626 ----min: -0.2973628342151642
b: 4.0
max error: 0.00012930601842825462
Client 19/40
max: 0.3317931890487671 ----min: -0.2997681498527527
b: 5.0
max error: 0.00011853675959013583
Client 20/40
max: 0.3236808776855469 ----min: -0.2967607378959656
b: 4.0
max error: 0.0002304840190216189
Client 21/40
max: 0.32591941952705383 ----min: -0.294381320476532
b: 4.0
max error: 0.00013883480363672618
Client 22/40
max: 0.3228277266025543 ----min: -0.2947721481323242
b: 5.0
max error: 0.00010252781496507702
Client 23/40
max: 0.3294161856174469 ----min: -0.2

KeyboardInterrupt: 

In [41]:
np.mean(Bits)

1712073.025

In [40]:
acc_q = np.array(global_acc_list)
acc_fed = np.array(global_acc_list_FedAvg)

# Base directory where you want to save the results
base_dir = 'C:/FedASi files'

# Prompt the user to input the folder name
#run_folder_name = input("Enter the folder name for this run: ")
run_folder_name = 'AQUILA_Noniid_K40_L5_T40'
run_folder_path = os.path.join(base_dir, run_folder_name)

# Create the directory if it doesn't exist
os.makedirs(run_folder_path, exist_ok=True)

# List of matrices to save and their respective filenames
matrices = {
    #'A_bar': A_bar,
    #'B_bar': B_bar,
    'Bits' : Bits[:, 0:39],
    #'percentage': percentage,
    #'I_M': I_M,
    #'B_tilde': B_tilde,
    #'Lambda': Lambda,
    'acc_q' : acc_q[ 0:39],
    'acc_fed': acc_fed[0:39]
    
    # Add more matrices here as needed
}

# Save each matrix to a .npy file in the specified folder
for name, matrix in matrices.items():
    file_name = os.path.join(run_folder_path, f'{name}.npy')
    np.save(file_name, matrix)
    print(f'{name} saved to {file_name}')


Bits saved to C:/FedASi files\AQUILA_Noniid_K40_L5_T40\Bits.npy
acc_q saved to C:/FedASi files\AQUILA_Noniid_K40_L5_T40\acc_q.npy
acc_fed saved to C:/FedASi files\AQUILA_Noniid_K40_L5_T40\acc_fed.npy
